## Treinamento Modelo

#### EDA Conjuntos de Treino

Avaliando cada um dos conjuntos de treino para entender o comportamento das variáveis de forma prévia.

In [7]:
# Libs:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pls
import seaborn as sns
import os

Vamos avaliar cada um dos conjuntos de treino de forma separada e depois podemos avaliar tudo junto, principalmente as **features** mais interessantes

In [199]:
# Looping os conj. de treino:
path = "C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\files\\treino\\"
df_full = pd.DataFrame()
for file in os.listdir(path):
    if file.__contains__("csv"):
        df = pd.read_csv(f"{path}{file}")
        print(f"Arquivo {file.replace('.csv', '')}: shape {df.shape}")
        
        # Concat dfs:
        df_full = pd.concat([df_full, df])
        
        del df
print(f"Full df: {df_full.shape}")

Arquivo treino_parte1: shape (100000, 10)
Arquivo treino_parte2: shape (100000, 10)
Arquivo treino_parte3: shape (100000, 10)
Arquivo treino_parte4: shape (100000, 10)
Arquivo treino_parte5: shape (100000, 10)
Arquivo treino_parte6: shape (77942, 10)
Full df: (577942, 10)


In [15]:
df_full.head(3)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."


In [20]:
df_full[df_full["userId"].isnull()]

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new


In [21]:
# Checando linearidade da base - por causa do historico:
df_full["userId"].duplicated().sum()

np.int64(0)

In [40]:
# Numero de linhas projetadas:
print(df_full.shape[0], "->", df_full["historySize"].sum())

577942 -> 8123951


Vamos tentar combinar duas técnicas, uma de session-based collaborative filtering e outra de content-based filtering, porém vamos tentar uma forma diferente da categoria. Para exemplificar o que tentaremos fazer, segue abaixo:

1. Criar um sistema de recomendação baseada em filtro colaborativo para cada sessão dos usuários para recomendar os N próximos artigos mais prováveis através das similaridades com outras sessões.
2. Depois rankear esses arquivos pela categoria deles, entretanto, aqui a categoria seria um pouco diferente, ao invés de usar categoria específcia vamos criar uma nova que é baseada em um score que iremos definir a partir de uma combinação entre algumas métricas interessantes.

### Criando o Score para as páginas

In [9]:
# Puxando as infos agragadas por pagina:
hists = []
times_pages = []
scroll_pp_pages = []
for i in range(len(df_full)):
    hists += df_full.iloc[i]["history"].split(", ")
    times_pages += df_full.iloc[i]["timeOnPageHistory"].split(", ")
    scroll_pp_pages += df_full.iloc[i]["scrollPercentageHistory"].split(", ")

In [10]:
print("Paginas:", len(hists))
print("Tempo na Pag:", len(times_pages))
print("Scroll Pag:", len(scroll_pp_pages))

Paginas: 1931607
Tempo na Pag: 1931607
Scroll Pag: 1931607


In [11]:
# Criando df para agregação:
df_paginas = (
    pd.DataFrame({
        "history": hists,
        "timeOnPageHistory": times_pages,
        "scrollPercentageHistory": scroll_pp_pages
    })
    .assign(
        history = lambda df_: df_.history.str.strip(),
        timeOnPageHistory = lambda df_: df_.timeOnPageHistory.astype(int),
        scrollPercentageHistory = lambda df_: df_.scrollPercentageHistory.astype(float)
    )
)

In [13]:
df_paginas.head()

,history,timeOnPageHistory,scrollPercentageHistory
0,c8aab885-433d-4e46-8066-479f40ba7fb2,20380,50.30
1,68d2039c-c9aa-456c-ac33-9b2e8677fba7,21184,18.18
2,13e423ce-1d69-4c78-bc18-e8c8f7271964,35438,16.46
3,bebdeb3e-1699-43e0-a1b8-989f5a6ab679,801396,78.74
4,f4b484a7-38f7-4246-be9e-4e2cd8373bcd,801396,78.74


In [14]:
df_paginas["history"].nunique()

182639

In [15]:
df_paginas.query("history == 'bc7ffaa4-e0e2-44ff-b8f5-24cc85bf5247'")

,history,timeOnPageHistory,scrollPercentageHistory


In [16]:
# Stats das Paginas:
df_paginas_stats = (
    df_paginas
    .assign(
        # Corrigindo infos
        # Scroll > 1:
        scrollPercentageHistory = lambda df_: np.where(df_.scrollPercentageHistory > 1, 1, df_.scrollPercentageHistory)
    )
    .groupby("history")
    .agg(
        n_app = ("timeOnPageHistory", "count"),
        avg_time = ("timeOnPageHistory", "mean"),
        avg_scroll = ("scrollPercentageHistory", "mean")
    )
    .reset_index()
    .assign(
        avg_time_norm = lambda df_: (df_.avg_time - df_.avg_time.min()) / (df_.avg_time.max() - df_.avg_time.min()),
        score = lambda df_: 0.7 * df_.n_app + 0.1 * df_.avg_time_norm + 0.2 * df_.avg_scroll,
        score_norm = lambda df_: ((df_.score - df_.score.min()) / (df_.score.max() - df_.score.min())) * 10 # min-max + new scale
    )
)

In [17]:
df_paginas_stats.sort_values(by = ["score_norm"], ascending = False).head()

,history,n_app,avg_time,avg_scroll,avg_time_norm,score,score_norm
149689,d2593c3d-2347-40d9-948c-b6065e8459a9,8806,81283.464002,1.0,0.009718,6164.400972,10.000000
135941,bf257382-74fb-4392-ad6a-143240e39f81,6102,84221.707473,1.0,0.010092,4271.601009,6.929083
176051,f6b5d170-48b9-4f8e-88d4-c84b6668f3bd,5456,81789.070565,1.0,0.009782,3819.400978,6.195424
116052,a36c98b5-f159-48f8-9f5a-1fc6ea9956c8,4539,78914.614453,1.0,0.009416,3177.500942,5.153993
22006,1f32787b-de2b-49be-8c20-ddaeae34cc22,4018,92743.733449,1.0,0.011178,2812.801118,4.562296


Vamos ver se esses scores fazem sentido, vamos puxar o top dez e ler a descrição para ter uma ideia.

In [81]:
path = "C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\itens\\itens\\"
df_desc_paginas = pd.DataFrame()
for file in os.listdir(path):
    if file.__contains__("csv"):
        df = pd.read_csv(f"{path}{file}")
        # Concat dfs:
        df_desc_paginas = pd.concat([df_desc_paginas, df])
        del df

In [82]:
df_desc_paginas.shape

(255603, 7)

In [83]:
df_desc_paginas.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


In [84]:
# Agregando descricoes aos stats das paginas:
df_desc_paginas = (
    df_desc_paginas
    .merge(df_paginas_stats, left_on = "page", right_on = "history")
)

In [85]:
top10 = df_desc_paginas.sort_values(by = "score_norm", ascending = False).head(10).reset_index(drop = True)
for i in range(10):
    print(f"""
    Página Top {i+1} - Score {top10.iloc[i]["score_norm"]}: 
    Título: {top10.iloc[i]["title"]}
    Legenda: {top10.iloc[i]["caption"]}
    """)


    Página Top 1 - Score 10.0: 
    Título: Anestesista é preso em flagrante por estupro de uma paciente que passava por cesárea no RJ
    Legenda: Giovanni Quintella Bezerra foi filmado por funcionários do Hospital da Mulher de São João de Meriti ao cometer crime. A defesa do anestesista disse que aguarda acesso à íntegra dos depoimentos para se manifestar. 
    

    Página Top 2 - Score 8.845023420518178: 
    Título: Diretor da Caixa Econômica Federal é encontrado morto na sede do banco em Brasília
    Legenda: Sérgio Ricardo Faustino Batista ocupava a diretoria de Controles Internos e Integridade da instituição, área que recebe denúncias sobre qualquer assunto, inclusive de assédio sexual, como as feitas contra o ex-presidente Pedro Guimarães. 
    

    Página Top 3 - Score 7.784167873031413: 
    Título: Filha é presa por golpe estimado em R$ 725 milhões contra a mãe; quadros renomados roubados foram recuperados
    Legenda: 
Segundo as investigações, a filha contratou pessoas 

Agora que temos um score para as noticías podemos começar a modelar a outra parte que seria a similaridade entre os usuários/sessões.

In [88]:
# Exportando scores das pagínas:
(
    df_desc_paginas
    [["page", "score_norm"]]
    .to_csv("C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\scores_paginas.csv", index = False)
)

#### Criando a parte de Collaborative Filtering

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import os

In [2]:
df_desc_paginas = pd.read_csv("C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\scores_paginas.csv")
df_desc_paginas.head()

,page,score_norm
0,13db0ab1-eea2-4603-84c4-f40a876c7400,0.000838
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,0.000439
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,4.984068
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,0.000838
4,9dff71eb-b681-40c7-ac8d-68017ac36675,0.002034


In [4]:
# Load dataset with memory optimization
dtypes = {
    "userId": "category",
    "userType": "category",
    "historySize": "int16",
}
# Looping os conj. de treino:
path = "C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\files\\treino\\"
df_full = pd.DataFrame()
for file in os.listdir(path):
    if file.__contains__("csv"):
        df = pd.read_csv(f"{path}{file}", dtype = dtypes)
        print(f"Arquivo {file.replace('.csv', '')}: shape {df.shape}")
        
        # Concat dfs:
        df_full = pd.concat([df_full, df.loc[df["historySize"] <= 30]])
        
        del df
print(f"Full df: {df_full.shape}")

# Convert lists stored as strings into actual lists
def parse_list(column, dtype):
    return df[column].apply(lambda x: list(map(dtype, x.split(","))))

Arquivo treino_parte1: shape (100000, 10)
Arquivo treino_parte2: shape (100000, 10)
Arquivo treino_parte3: shape (100000, 10)
Arquivo treino_parte4: shape (100000, 10)
Arquivo treino_parte5: shape (100000, 10)
Arquivo treino_parte6: shape (77942, 10)
Full df: (514140, 10)


In [5]:
df_full.head(2)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [6]:
def create_sparse_matrix(df):
    user_to_index = {user: i for i, user in enumerate(df["userId"].unique())}
    page_to_index = {page: i for i, page in enumerate(set(",".join(df["history"]).split(",")))}
    page_to_index = {key.strip(): value for key, value in page_to_index.items()}
    
    num_users = len(user_to_index)
    num_pages = len(page_to_index)

    row, col, data = [], [], []
    
    for _, row_data in df.iterrows():
        user_idx = user_to_index[row_data["userId"]]
        for page in row_data["history"].split(","):
            if page in page_to_index:
                page_idx = page_to_index[page]
                row.append(user_idx)
                col.append(page_idx)
                score = df_desc_paginas.loc[df_desc_paginas["page"] == page]["score_norm"].values[0]
                data.append(score) #time_on_page * scroll)  # Interaction score

    sparse_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_pages), dtype=np.float32)
    
    return sparse_matrix, user_to_index, page_to_index

In [ ]:
sparse_matrix, user_to_index, page_to_index = create_sparse_matrix(df_full)

In [ ]:
sparse_matrix

In [ ]:
def compute_cosine_similarity(sparse_matrix):
    return cosine_similarity(sparse_matrix, dense_output=False)  # Keep it sparse

In [ ]:
user_similarity = compute_cosine_similarity(sparse_matrix)

In [ ]:
def hybrid_recommendation(user_id, top_n=5, alpha=0.6):
    if user_id not in user_to_index:
        return []

    user_idx = user_to_index[user_id]

    # Find the most similar users
    similar_users = np.argsort(-user_similarity[user_idx].toarray().flatten())[1:6]  # Skip itself

    # Compute user-based recommendations
    user_recs = np.mean(sparse_matrix[similar_users].toarray(), axis=0)

    # Compute item-based recommendations
    user_history = df_full[df_full["userId"] == user_id]["history"].values[0].split(",")
    # item_recs = np.zeros(sparse_matrix.shape[1])

    #for page in user_history:
    #    if page in page_to_index:
    #        item_recs += sparse_matrix[:, page_to_index[page]].toarray().flatten()

    # Combine scores (User CF + Item CF)
    final_scores = alpha * user_recs + (1 - alpha) #* item_recs
    recommended_indices = np.argsort(final_scores)[::-1][:top_n]

    # Convert back to page IDs
    index_to_page = {v: k for k, v in page_to_index.items()}
    return [index_to_page[idx] for idx in recommended_indices]

In [ ]:
# Example usage
recommended_pages = hybrid_recommendation("f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069")
print(recommended_pages)

In [222]:
df_full["userId"].values[0]

'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069'

In [224]:
df_val = pd.read_csv("C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\validacao.csv")

In [226]:
df_val.head()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8'\n '01c...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85'\n '835...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6'\n 'b8e...,[1660548813953 1660572329731 1660594848200]


#### ALS

In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
import implicit

C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Fase 5\Datathon\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\files\\treino\\treino_parte1.csv")

In [7]:
df.columns

Index(['userId', 'userType', 'historySize', 'history', 'timestampHistory',
       'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory',
       'pageVisitsCountHistory', 'timestampHistory_new'],
      dtype='object')

In [ ]:
df_t = pd.DataFrame()
for i in range(len(df)):
    # user Id
    userId = df["userId"].values[i]
    d_t = {"userId": userId}
    for _col in ['history', 'timestampHistory', 'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory']:
        
        # ajustando valores:
        values = df[_col].values[i].split(",")
        d_t[_col] = values
        
    df_t = pd.concat([df_t, pd.DataFrame(d_t)])

In [ ]:
df_t.head()

In [17]:
df_t = (
    df_t
    .assign(
        numberOfClicksHistory = lambda df_: df_.numberOfClicksHistory.astype(np.int32),
        scrollPercentageHistory = lambda df_: df_.scrollPercentageHistory.astype(np.float32),
        timeOnPageHistory = lambda df_: df_.timeOnPageHistory.astype(np.int32)
    )
)

In [18]:
df_t["interaction_score"] = (
    df_t["timeOnPageHistory"] * 0.5 +  # More time on page = higher interest
    df_t["scrollPercentageHistory"] * 0.3 +  # More scrolling = higher engagement
    df_t["numberOfClicksHistory"] * 0.2  # More clicks = higher interaction
)

In [20]:
user_mapping = {u: i for i, u in enumerate(df_t["userId"].unique())}
item_mapping = {i: j for j, i in enumerate(pd.unique(",".join(df_t["history"]).split(", ")))}

C:\Users\lucaa\AppData\Local\Temp\ipykernel_16500\1672575828.py:2: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  item_mapping = {i: j for j, i in enumerate(pd.unique(",".join(df_t["history"]).split(", ")))}


In [23]:
df_t["user_index"] = df_t["userId"].map(user_mapping)

# 🔹 Transform history column (which is a list) into rows
exploded_df = df_t.explode(["history", "interaction_score"])
exploded_df["item_index"] = exploded_df["history"].map(item_mapping)

In [36]:
# 🔹 Create Sparse User-Item Matrix
rows = exploded_df["user_index"].values
cols = exploded_df["item_index"].fillna(0).astype(np.int32).values

In [39]:
data = exploded_df["interaction_score"].astype(np.int32).values

In [40]:
data

array([10220, 10605, 17732, ...,  5027, 20010,  5003],
      shape=(347450,), dtype=int32)

In [41]:
user_item_matrix = coo_matrix((data, (rows, cols)))

In [42]:
user_item_matrix

<COOrdinate sparse matrix of dtype 'int32'
	with 347450 stored elements and shape (23582, 55133)>

In [43]:
# 🔹 Train ALS Model
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(user_item_matrix)

C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Fase 5\Datathon\venv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Fase 5\Datathon\venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.006756782531738281 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.26it/s]


In [44]:
# 🔹 Function to Recommend News for a User
def recommend_news(user_id, top_n=5):
    if user_id not in user_mapping:
        return "User not found"
    
    user_idx = user_mapping[user_id]
    recommended_items = als_model.recommend(user_idx, user_item_matrix, N=top_n)
    
    recommended_news = [list(item_mapping.keys())[list(item_mapping.values()).index(item)] for item, score in recommended_items]
    return recommended_news

In [46]:
user_to_recommend = "f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069"

In [51]:

# 🔹 Convert userId and history (news IDs) to integer indices
user_mapping = {u: i for i, u in enumerate(df["userId"].unique())}
item_mapping = {i: j for j, i in enumerate(df["history"].unique())}

df["user_index"] = df["userId"].map(user_mapping)
df["item_index"] = df["history"].map(item_mapping)

# 🔹 Create Sparse User-Item Matrix (CSR Format)
rows = df["user_index"].values
cols = df["item_index"].values
data = df["interaction_score"].values

# Convert to CSR matrix
user_item_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_mapping), len(item_mapping)))

# 🔹 Train ALS Model
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(user_item_matrix.T)  # Transposing for correct dimensions

# 🔹 Function to Recommend News for a User
def recommend_news(user_id, top_n=5):
    if user_id not in user_mapping:
        return "User not found"
    
    user_idx = user_mapping[user_id]
    recommended_items = als_model.recommend(user_idx, user_item_matrix, N=top_n)
    
    recommended_news = [list(item_mapping.keys())[list(item_mapping.values()).index(item)] for item, score in recommended_items]
    return recommended_news

# 🔹 Example: Recommend 5 news articles for a user
user_to_recommend = "f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069"
print(recommend_news(user_to_recommend))


C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Fase 5\Datathon\venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.02097153663635254 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.43it/s]


ValueError: user_items must contain 1 row for every user in userids